In [12]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.stats import zscore
from collections import Counter
import numpy as np

In [13]:
df =  pd.read_csv("movies.csv")


In [14]:
display(df.columns)

Index(['id', 'title', 'genres', 'original_language', 'overview', 'popularity',
       'production_companies', 'release_date', 'budget', 'revenue', 'runtime',
       'status', 'tagline', 'vote_average', 'vote_count', 'credits',
       'keywords', 'poster_path', 'backdrop_path', 'recommendations'],
      dtype='object')

In [15]:
# Converter a coluna para datetime (se ainda não estiver)
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Criar as colunas release_year e release_month
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month

In [16]:
# Substituir zeros por 1
df['budget'] = df['budget'].replace(0, 1)
df['revenue'] = df['revenue'].replace(0, 1)

# Aplicar transformação logarítmica e criar colunas novas
df['budget_log'] = np.log(df['budget'])
df['revenue_log'] = np.log(df['revenue'])

c:\Users\thalyson\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [17]:
# --- 3. Tratar gêneros: separar por '-' e agrupar gêneros raros em 'Outros' ---
df['genres'] = df['genres'].fillna('')  # preencher nulos com string vazia
df['genres'] = df['genres'].apply(lambda x: x.split('-') if x else [])

contagem = Counter([genero for sublist in df['genres'] for genero in sublist])
generos_raros = [g for g, count in contagem.items() if count < 900]

def substituir_raros(lista):
    return [g if g not in generos_raros else 'Outros' for g in lista]

df['genres'] = df['genres'].apply(substituir_raros)

# --- 4. Transformar genres em dummies ---
mlb = MultiLabelBinarizer()
genres_dummies = mlb.fit_transform(df['genres'])
df_genres = pd.DataFrame(genres_dummies, columns=mlb.classes_, index=df.index)


display(df.columns)

Index(['id', 'title', 'genres', 'original_language', 'overview', 'popularity',
       'production_companies', 'release_date', 'budget', 'revenue', 'runtime',
       'status', 'tagline', 'vote_average', 'vote_count', 'credits',
       'keywords', 'poster_path', 'backdrop_path', 'recommendations',
       'release_year', 'release_month', 'budget_log', 'revenue_log'],
      dtype='object')

In [18]:
colunas_uteis = ['genres',
    'original_language',
    'popularity',
    'budget_log',
    'revenue_log',
    'runtime',
    'vote_average',
    'vote_count',
    'release_year',
    'release_month'
]